In [1]:
# -*- coding: utf-8 -*-
from sys import path
path.append('../')
import hsim.core.pymulate as pym
from hsim.core.chfsm import CHFSM, Transition, State
import pandas as pd
import numpy as np
from simpy import AnyOf
from copy import deepcopy
from random import choices,seed,normalvariate, expovariate
from hsim.core.stores import Store, Box       
from scipy import stats
import dill
import hsim.core.utils as utils

c:\Users\Utente\GitHub\hsim\hsim\core\core.py:57: UserWarning: Bypassing "None" callbakcs
  warn('Bypassing "None" callbakcs')


In [2]:

import numpy as np
import torch
import torch.nn as nn
import random
from collections import deque

class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )
    
    def forward(self, x):
        return self.network(x)

import random
from collections import deque

class DQNAgent:
    def __init__(self, state_dim, action_dim):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.memory = deque(maxlen=10000)
        self.gamma = 0.99
        self.epsilon =1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.model = DQN(state_dim, action_dim)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
    
    def act(self, state):
        if random.random() <= self.epsilon:
            return random.randrange(self.action_dim)
        state = torch.FloatTensor(state).unsqueeze(0)
        action_values = self.model(state)
        return torch.argmax(action_values).item()
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward + (self.gamma * torch.max(self.model(torch.FloatTensor(next_state).unsqueeze(0))) * (1 - done))
            current_q = self.model(torch.FloatTensor(state).unsqueeze(0))[0][action]
            loss = (current_q - target) ** 2
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

In [113]:
class Generator(pym.Generator):
    def __init__(self,env,name=None,serviceTime=2,serviceTimeFunction=None):
        super().__init__(env,name,serviceTime,serviceTimeFunction)
        self.count = 0
    def createEntity(self):
        self.count += 1
       #print(self.count)
        # return Entity()
        e = Entity()
        #e.serviceTime = dict()
        e.serviceTime['front'] = 10.52
        e.serviceTime['drill'] = choices([3.5, 8.45, 9.65, 11.94],weights=[5,30,30,35])[0] ## Insert the possibility of skipping this stage
        e.serviceTime['robot'] = choices([0, 81, 105, 108 ,120],weights=[91,3,2,2,2])[0]
        # e.serviceTime['camera'] = choices([3,9,12,18,24],weights=[2,3,1,2,2])[0]
        e.serviceTime['camera'] = 3.5+expovariate(1/7.1)
        e.serviceTime['back'] = choices([3.5,10.57],weights=[0.1,0.9])[0]
        # e.serviceTime['press'] = choices([3,9,15])[0]
        if e.serviceTime['back']>0:
            e.serviceTime['press'] = 3.5+expovariate(1/9.5)
        else:
            e.serviceTime['press'] = 3.5
        e.serviceTime['manual'] = max(np.random.normal(9.2,1),0)
        return e
        
class Entity:
    def __init__(self,ID=None):
        self.ID = ID
        self.rework = False
        self.serviceTime = dict()
        # self.pt['M3'] = 1
    @property
    def require_robot(self):
        if self.serviceTime['robot']>0:
            return True
        else:
            return False
    @property
    def ok(self):
        return not (self.rework and self.require_robot)
    def done(self):
        self.rework = False
                
class LabServer(pym.Server):
    def __init__(self,env,name=None,serviceTime=None,serviceTimeFunction=None):
        self.controller = None
        # serviceTime = 10
        super().__init__(env,name,serviceTime,serviceTimeFunction)
    def calculateServiceTime(self,entity=None,attribute='serviceTime'):
        if not entity.ok:
            return 3.5 ### qua andrà messo 10/20 volte maggiore degli altri processing time?
        else:
            print("service time")
            print(super().calculateServiceTime(entity,attribute))
            return super().calculateServiceTime(entity,attribute)
    def completed(self):
        if self.var.entity.ok:
            self.controller.Messages.put(self.name)
    T2=Transition(pym.Server.Working, pym.Server.Blocking, lambda self: self.env.timeout(self.calculateServiceTime(self.var.entity)), action = lambda self: self.completed())
    T3=Transition(pym.Server.Blocking, pym.Server.Starving, lambda self: self.Next.put(self.var.entity),action=lambda self: [self.var.request.confirm(), self.sm.var.entity.done() if self.sm._name=='robot' else None])

class Terminator(pym.Terminator):
    def __init__(self, env, capacity=np.inf):
        super().__init__(env, capacity)
        self.controller = None
        self.register = list()
    def completed(self):
        if not self.trigger.triggered:
            self.trigger.succeed()
    def put(self,item):
        self.register.append(self._env.now)
        self.controller.Messages.put('terminator')
        return super().put(item)
    def subscribe(self,item):
        self.register.append(self._env.now)
        self.controller.Messages.put('terminator')
        return super().subscribe(item)
class Gate(CHFSM):
    # #AGGIUNTA
    # def _init_(self):
    #     self.dqn_agent = DQNAgent(state_dim=3, action_dim=50) #dimsnesioni da cambiare
    # def get_current_state(self):   
    # # da definire col metodo che si vuole usare
    # def action_to_request(self, action):
    # #DA DEFINIRE 
    #FINE AGGIUNTA
    def __init__(self,env):
        self.real = True
        self.capacity = 30
        self.lab = None
        self.initialWIP = 12
        self.targetWIP = 12
        self.request = None
        self.message = env.event()
        self.WIP = 0
        self.WIPlist = list()
        super().__init__(env)
    def build(self):
        self.Store = pym.Store(self.env,self.capacity)
        self.Messages = pym.Store(self.env)
    def put(self,item):
        return self.Store.put(item)
    class Loading(State):
        def _do(self):
            # print('Load: %d' %self.sm.initialWIP)
            self.sm.initialWIP -= 1
            self.fw()
    class Waiting(State):
        initial_state = True
        def _do(self):
            self.sm.message = self.Messages.subscribe()
            if self.sm.initialWIP > 0:
                self.initial_timeout = self.env.timeout(1)
            else:
                self.initial_timeout = self.env.event()
    class Forwarding(State):
        def _do(self):
            self.message.confirm()
            if self.message.value == 'terminator':
                self.sm.WIP -= 1
                self.sm.WIPlist.append([self.env.now,self.WIP])
            self.FIFO()
            self.CONWIP()
    def CONWIP(self):
        if self.message.value == 'terminator':
            self.fw()
    def FIFO(self):
        pass 
    def fw(self):        
        if self.request is None:
            self.request = self.Store.get()
        #INIZIO NUOVO CODICE
            #Scheduling Agent
        # Ottieni lo stato corrente del sistema per l'agente DQN
        # current_state = self.get_current_state()
        # Utilizza l'agente DQN per selezionare l'azione basata sullo stato corrente
        # #action = self.dqn_agent.act(current_state)
        
        # Mappa l'azione selezionata in una richiesta effettiva
        #scheduled_request = self.action_to_request(action)
        
        #if scheduled_request is not None:
        #    self.request = scheduled_request
        #else:
            # Nessuna richiesta programmata, gestire il caso
        #    pass

    # Procedere con la gestione della richiesta programmata
              
        # QUA RICOMINCIA IL VECCHIO CODICE
            #self.request = self.Store.get() --> da inserire come fallback ma aggiungendo un controllo
        if self.request is not None:
            try:
                self.Next.put(self.request.value)
                self.request = None
                self.WIP += 1
                self.WIPlist.append([self.env.now,self.WIP])
        
            except:
                pass
            # print('Empty at %s' %self.env.now)
    T0 = Transition(Waiting,Loading,lambda self: self.initial_timeout)
    T1 = Transition(Waiting,Forwarding,lambda self: self.sm.message)
    T2 = Transition(Loading,Waiting,None)
    T3 = Transition(Forwarding,Waiting,None)

class Router(pym.Router):
    def __deepcopy(self,memo):
        super().deepcopy(self,memo)
    def __init__(self, env, name=None):
        super().__init__(env, name)
        self.var.requestOut = []
        self.var.sent = []
        self.putEvent = env.event()
    def build(self):
        self.Queue = Box(self.env)
    def condition_check(self,item,target):
        return True
    def put(self,item):
        if self.putEvent.triggered:
            self.putEvent.restart()
        self.putEvent.succeed()
        return self.Queue.put(item)
    class Sending(State):
        initial_state = True
        def _do(self):
            self.sm.putEvent.restart()
            self.sm.var.requestIn = self.sm.putEvent
            self.sm.var.requestOut = [item for sublist in [[next.subscribe(item) for next in self.sm.Next if self.sm.condition_check(item,next)] for item in self.sm.Queue.items] for item in sublist]
            if self.sm.var.requestOut == []:
                self.sm.var.requestOut.append(self.sm.var.requestIn)
    S2S2 = Transition(Sending,Sending,lambda self:AnyOf(self.env,self.var.requestOut),condition=lambda self:self.var.requestOut != [])
    def action2(self):
        self.Queue._trigger_put(self.env.event())
        if not hasattr(self.var.requestOut[0],'item'):
            return
        for request in self.var.requestOut:
            if not request.item in self.Queue.items:
                request.cancel()
                continue
            if request.triggered:
                if request.check():
                    request.confirm()
                    self.Queue.forward(request.item)
                    continue
    S2S2._action = action2
'''  
from pymulate import RouterNew
class Router(RouterNew):
    def __init__(self, env, name=None):
        capacity=1
        super().__init__(env, name,capacity)
'''
class RobotSwitch1(Router):
    def condition_check(self, item, target):
        if item.require_robot:
            item.rework = True
        if item.require_robot and target.name == 'convRobot1S':
            return True
        elif not item.require_robot and target.name != 'convRobot1S':
            return True
        else:
            return False
            
class RobotSwitch2(Router):
    def condition_check(self, item, target):
        if len(target.Next)<2:
            item.rework = False
            return True
        else:
            item.rework = True
            return False    

class CloseOutSwitch(Router):
    def condition_check(self, item, target):
        if item.ok and type(target) == Terminator:
            return True
        elif not item.ok and type(target) != Terminator:
            return True
        else:
            return False
        
# class Conveyor(pym.ParallelServer):
#     def __init__(self,env,name=None,serviceTime=None,serviceTimeFunction=None,capacity=1):
#         self._capacity = capacity
#         serviceTime = capacity*3.5
#         super().__init__(env,name,serviceTime,serviceTimeFunction,capacity)
class Conveyor(pym.Conveyor):
    def __init__(self,env,name=None,capacity=3):
        super().__init__(env,name,capacity,0.75)
        
def newDT():
    lab = globals()['lab']
    deepcopy(lab)

def newEntity(): #viene usata all interno di batch create
    e = Entity()
    e.serviceTime['front'] = 10.52
    e.serviceTime['drill'] = 40 #choices([3.5, 8.45, 9.65, 11.94],weights=[5,30,30,35])[0]
    e.serviceTime['robot'] = choices([0, 81, 105, 108 ,120],weights=[91,3,2,2,2])[0]
    e.serviceTime['camera'] = 3.5+expovariate(1/7.1)
    e.serviceTime['back'] = choices([3.5,10.57],weights=[0.1,0.9])[0]
    if e.serviceTime['back']>0:
        e.serviceTime['press'] = 3.5+expovariate(1/9.5)
    else:
        e.serviceTime['press'] = 3.5
    e.serviceTime['manual'] = max(np.random.normal(9.2,1),0)
    return e

def batchCreate(seed=1,numJobs=10):
    np.random.seed(seed)
    jList = []
    while len(jList)<numJobs:
        e=newEntity()
        num = round(np.random.triangular(1,numJobs/2,numJobs))
        # print(num)
        for i in range(num):
            jList.append(deepcopy(e))
            if len(jList)>=numJobs:  # l'ho aggiunto io per evitare che si creino più entità di quelle richieste
                break
    print("questa è la job list")
    for i, entity in enumerate(jList):
       print(f"Entity {i+1} service times: {entity.serviceTime}")
  
    return jList

class Lab:
    def __init__(self):
        conveyTime = 6
        self.env = pym.Environment() #crea l'ambiente
        self.g = Generator(self.env) #genera un nuovo pezzo
        self.gate = Gate(self.env) #da togliere ma ci sarà bisogno di modifiche #gestisce l'ingresso dei pezzi
        # DR= despaching rule OR=order release
        # self.conv1 = Conveyor(self.env,capacity=3)
        self.conv1S = pym.Server(self.env,serviceTime=conveyTime) 
        self.conv1Q = pym.Queue(self.env,capacity=2)
        self.front = LabServer(self.env,'front')
        # self.conv2 = Conveyor(self.env,capacity=3)
        self.conv2S = pym.Server(self.env,serviceTime=conveyTime)
        self.conv2Q = pym.Queue(self.env,capacity=2)
        self.drill = LabServer(self.env,'drill')
        # self.conv3 = Conveyor(self.env,capacity=3)
        self.conv3S = pym.Server(self.env,serviceTime=conveyTime)
        self.conv3Q = pym.Queue(self.env,capacity=2)

        
        self.switch1 = RobotSwitch1(self.env)
        # self.convRobot1 = Conveyor(self.env,'convRobot1',capacity=3)
        self.convRobot1S = pym.Server(self.env,serviceTime=conveyTime,name='convRobot1S')
        self.convRobot1Q = pym.Queue(self.env,capacity=2)

        # self.bridge = Conveyor(self.env,capacity=3)
        self.bridgeS = pym.Server(self.env,serviceTime=conveyTime)
        self.bridgeQ = pym.Queue(self.env,capacity=2)

        # self.convRobot2 = Conveyor(self.env,'convRobot2',capacity=3)
        self.convRobot2S = pym.Server(self.env,serviceTime=conveyTime)
        self.convRobot2Q = pym.Queue(self.env,capacity=2)

        self.switch2 = RobotSwitch2(self.env)
        # self.convRobot3 = Conveyor(self.env,capacity=3)
        self.convRobot3S = pym.Server(self.env,serviceTime=conveyTime)
        self.convRobot3Q = pym.Queue(self.env,capacity=2)

        self.robot = LabServer(self.env,'robot')
        # self.convRobotOut = Conveyor(self.env,capacity=3)
        self.convRobotOutS = pym.Server(self.env,serviceTime=conveyTime)
        self.convRobotOutQ = pym.Queue(self.env,capacity=2)
        # self.conv5 = Conveyor(self.env,capacity=3)
        self.conv5S = pym.Server(self.env,serviceTime=conveyTime)
        self.conv5Q = pym.Queue(self.env,capacity=2)

        self.camera = LabServer(self.env,'camera')
        # self.conv6 = Conveyor(self.env,capacity=3)
        self.conv6S = pym.Server(self.env,serviceTime=conveyTime)
        self.conv6Q = pym.Queue(self.env,capacity=2)

        self.back = LabServer(self.env,'back')
        # self.conv7 = Conveyor(self.env,capacity=3)
        self.conv7S = pym.Server(self.env,serviceTime=conveyTime)
        self.conv7Q = pym.Queue(self.env,capacity=2)

        self.press = LabServer(self.env,'press')
        # self.conv8 = Conveyor(self.env,capacity=3)
        self.conv8S = pym.Server(self.env,serviceTime=conveyTime)
        self.conv8Q = pym.Queue(self.env,capacity=2)

        self.manual = LabServer(self.env,'manual')
        self.outSwitch = CloseOutSwitch(self.env)
        self.terminator = Terminator(self.env)
        
        self.g.Next = self.gate
        self.gate.Next = self.conv1S
        
        # self.conv1.Next = self.front
        self.conv1S.Next = self.conv1Q
        self.conv1Q.Next = self.front

        self.front.Next = self.conv2S
        # self.conv2.Next = self.drill
        self.conv2S.Next = self.conv2Q
        self.conv2Q.Next = self.drill
        self.drill.Next = self.conv3S
        self.conv3S.Next = self.conv3Q
        self.conv3Q.Next = self.switch1
        # self.conv3.Next = self.switch1
        
        self.switch1.Next = [self.convRobot1S,self.bridgeS]
        self.convRobot1S.Next = self.convRobot1Q
        self.convRobot1Q.Next = self.switch2

        self.switch2.Next = [self.convRobot2S,self.convRobot3S]
        self.convRobot2S.Next = self.convRobot2Q
        self.convRobot2Q.Next = self.robot

        self.convRobot3S.Next = self.convRobot3Q
        self.convRobot3Q.Next = self.convRobotOutS

        self.robot.Next = self.convRobotOutS
        self.convRobotOutS.Next = self.convRobotOutQ

        self.convRobotOutQ.Next = self.conv5S
        self.bridgeS.Next = self.bridgeQ
        self.bridgeQ.Next = self.conv5S

        
        self.conv5S.Next = self.conv5Q
        self.conv5Q.Next = self.camera

        self.camera.Next = self.conv6S
        self.conv6S.Next = self.conv6Q
        self.conv6Q.Next = self.back

        self.back.Next = self.conv7S
        self.conv7S.Next = self.conv7Q
        self.conv7Q.Next = self.press

        self.press.Next = self.conv8S
        self.conv8S.Next = self.conv8Q
        self.conv8Q.Next = self.manual

        self.manual.Next = self.outSwitch
        self.outSwitch.Next = [self.conv1S,self.terminator]
        
        for x in [self.front,self.drill,self.robot,self.camera,self.back,self.press,self.manual]:
            x.controller = self.gate
        self.terminator.controller = self.gate
        self.gate.lab = self

    def run(self,Tend):
        self.env.run(Tend)
        # return pd.DataFrame(self.env.state_log)
        return self.env.state_log





# import sys
# sys.path.insert(0,'C:/Users/Lorenzo/Dropbox (DIG)/Tesisti/Giovanni Zanardo/Python')
# from Foresighted_DT_function import BN_detection

# BN_detection(lab.env.state_log,0,lab.env.now)

# class Result:
#     def __init__(self,time,BN,OR,DR,production,arrivals,WIPlist,BNlist,state_log):
#         self.time=time
#         self.BN=BN
#         self.OR=OR
#         self.DR=DR
#         self.production=production
#         self.arrivals=arrivals
#         self.WIPlist = WIPlist
#         self.state_log=state_log
#         self.BNlist = BNlist
#     @property
#     def avgWIP(self):
#         integral = 0.0
#         prev_time = None
#         prev_value = None
#         # Iterate over the data points and compute the integral
#         for time, value in self.WIPlist:
#             if prev_time is not None and prev_value is not None:
#                 dt = time - prev_time
#                 integral += prev_value * dt
#             prev_time = time
#             prev_value = value
#         return integral/time
#     @property
#     def productivity(self):
#         return (3600/pd.DataFrame(self.arrivals).diff()).describe()
#     @property
#     def CI(self):
#         prod = self.productivity
#         tinv = stats.t.ppf(1-0.05/2, prod[0][0])
#         return prod[0][1] - prod[0][2]*tinv/np.sqrt(prod[0][0]), prod[0][1] + prod[0][2]*tinv/np.sqrt(prod[0][0])





In [114]:
lab=Lab()
lab.gate.Store.items = batchCreate(0,numJobs=10) #batchedExp
print(lab.gate.Store.items)
df1=pd.DataFrame(lab.gate.Store.items)


lab.run(3600)

df = pd.DataFrame(lab.env.state_log, columns=["Resource","ResourceName","State","StateName","Entity","?","timeIn","timeOut"])

df = df.loc[df.ResourceName.isin(["front","drill","robot","camera","back","press","manual"])]

fig = utils.createGantt(df)

fig.show()

makespan=df.timeOut.max()-df.timeIn.min()
print(makespan)
selected_columns = df[["Entity", "State"]]
print(selected_columns)


df1 = pd.read_excel('batch.xlsx')
df = pd.read_excel('output.xlsx')

# Crea un oggetto ExcelWriter
with pd.ExcelWriter('combined_output.xlsx') as writer:
    # Scrivi ciascun DataFrame in un foglio diverso
    df1.to_excel(writer, sheet_name='Foglio1', index=False)
    df.to_excel(writer, sheet_name='Foglio2', index=False)

6
5
questa è la job list
Entity 1 service times: {'front': 10.52, 'drill': 40, 'robot': 0, 'camera': 11.478560310698168, 'back': 10.57, 'press': 28.596513798506773, 'manual': 10.964052345967664}
Entity 2 service times: {'front': 10.52, 'drill': 40, 'robot': 0, 'camera': 11.478560310698168, 'back': 10.57, 'press': 28.596513798506773, 'manual': 10.964052345967664}
Entity 3 service times: {'front': 10.52, 'drill': 40, 'robot': 0, 'camera': 11.478560310698168, 'back': 10.57, 'press': 28.596513798506773, 'manual': 10.964052345967664}
Entity 4 service times: {'front': 10.52, 'drill': 40, 'robot': 0, 'camera': 11.478560310698168, 'back': 10.57, 'press': 28.596513798506773, 'manual': 10.964052345967664}
Entity 5 service times: {'front': 10.52, 'drill': 40, 'robot': 0, 'camera': 11.478560310698168, 'back': 10.57, 'press': 28.596513798506773, 'manual': 10.964052345967664}
Entity 6 service times: {'front': 10.52, 'drill': 40, 'robot': 0, 'camera': 11.478560310698168, 'back': 10.57, 'press': 28.59

c:\Users\Utente\GitHub\hsim\hsim\core\utils.py:64: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





0 days 00:59:57.826149041
                                               Entity  \
4                                                None   
7                                                None   
20                                               None   
25                                               None   
28                                               None   
...                                               ...   
18991  <__main__.Entity object at 0x0000022704F92610>   
18992  <__main__.Entity object at 0x0000022704F92610>   
18996  <__main__.Entity object at 0x0000022704F92610>   
19006  <__main__.Entity object at 0x00000227051B2790>   
19013  <__main__.Entity object at 0x00000227051B2790>   

                                            State  
4      <Starving (State) object at 0x22705a1aed0>  
7      <Starving (State) object at 0x22705a1b910>  
20     <Starving (State) object at 0x227041d8590>  
25     <Starving (State) object at 0x227058f5750>  
28     <Starving (State) obje

In [59]:
njobs = 10
jList1 = []
while len(jList1)<njobs:
    e=newEntity()
    num = round(np.random.triangular(1,njobs/2,njobs))
    for i in range(num):
        jList1.append(deepcopy(e))
# print("questa è la job list")
# print(jList1)

/n
{'front': 10.52, 'drill': 40, 'robot': 0, 'camera': 8.014912922560246, 'back': 10.57, 'press': 6.225301806275532, 'manual': 10.058754152991916}
/n
{'front': 10.52, 'drill': 40, 'robot': 0, 'camera': 15.14941935767573, 'back': 10.57, 'press': 14.230520522375148, 'manual': 8.917864858396081}


In [71]:
for i in range(njobs):
    print(jList1[i].serviceTime)


{'front': 10.52, 'drill': 40, 'robot': 0, 'camera': 8.014912922560246, 'back': 10.57, 'press': 6.225301806275532, 'manual': 10.058754152991916}
{'front': 10.52, 'drill': 40, 'robot': 0, 'camera': 8.014912922560246, 'back': 10.57, 'press': 6.225301806275532, 'manual': 10.058754152991916}
{'front': 10.52, 'drill': 40, 'robot': 0, 'camera': 8.014912922560246, 'back': 10.57, 'press': 6.225301806275532, 'manual': 10.058754152991916}
{'front': 10.52, 'drill': 40, 'robot': 0, 'camera': 8.014912922560246, 'back': 10.57, 'press': 6.225301806275532, 'manual': 10.058754152991916}
{'front': 10.52, 'drill': 40, 'robot': 0, 'camera': 15.14941935767573, 'back': 10.57, 'press': 14.230520522375148, 'manual': 8.917864858396081}
{'front': 10.52, 'drill': 40, 'robot': 0, 'camera': 15.14941935767573, 'back': 10.57, 'press': 14.230520522375148, 'manual': 8.917864858396081}
{'front': 10.52, 'drill': 40, 'robot': 0, 'camera': 15.14941935767573, 'back': 10.57, 'press': 14.230520522375148, 'manual': 8.917864858